In [216]:
from cot import Collection
import pandas as pd
import numpy as np
import krippendorff
import random
import scipy
from scipy import stats
pd.set_option('display.max_rows', 200)
pd.options.display.float_format = "{:.2f}".format
%precision 2

'%.2f'

In [220]:
coll = Collection.from_json("/home/kon/work/ThoughtSource/libs/cot/cot/datasets/thoughtsource/thoughtsource_33_paper.json")

In [223]:
# from cot.evaluate import compare_evaluation_difference
# compare_evaluation_difference(coll)

In [224]:
df = coll.collection_to_dataframe()

In [225]:
df.head(1)

,dataset,split,id,type,answer_label,prompt,instruction,cot_trigger,answer_id,answer_from_choices,correct_answer,model
0,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,multiplechoice,E,None_None,None,None,20897a78-9ce5-45f4-8f5f-6e50b5c8fd65,E,True,command-xlarge-nightly


In [226]:
df[df['correct_answer'].isna()]["answer_from_choices"].value_counts()

Series([], Name: answer_from_choices, dtype: int64)

In [227]:
df[df['answer_from_choices'].isna()]["correct_answer"].value_counts()

Series([], Name: correct_answer, dtype: int64)

In [228]:
# check if those items are from StrategyQA

In [229]:
df[(df['answer_from_choices'].isna()) & (df['correct_answer'] == False)]

,dataset,split,id,type,answer_label,prompt,instruction,cot_trigger,answer_id,answer_from_choices,correct_answer,model


In [152]:
# ammount of NaN anwers per model. These are inconclusive answers given, and the evaluation code does not take them into account.
df.groupby(["model"]).correct_answer.apply(lambda x: x.isnull().sum())

model
command-xlarge-nightly     18
flan-T5-xxl                28
gpt-3.5-turbo             105
gpt-4                      63
text-davinci-002           55
text-davinci-003           78
Name: correct_answer, dtype: int64

# Accuracy

In [230]:
def calculate_confidence_intervals(dataframe, index_col, columns_col, values_col, confidence_level=0.95):
    df = dataframe.copy()
    df = df.dropna(subset=['correct_answer'])
    df["correct_answer"] = df["correct_answer"].fillna(np.nan)
    from scipy.stats import t
    # Calculate mean accuracy scores
    acc = df.groupby([columns_col, index_col]).mean().pivot_table(index=index_col, columns=columns_col, values=values_col)

    # Calculate standard error for each group
    standard_errors = df.groupby([columns_col, index_col]).sem().pivot_table(index=index_col, columns=columns_col, values=values_col)

    # Calculate standard deviations for each group
    standard_deviations = df.groupby([columns_col, index_col]).std().pivot_table(index=index_col, columns=columns_col, values=values_col)

    # Calculate degrees of freedom for each group
    degrees_freedom = df.groupby([columns_col, index_col]).count().pivot_table(index=index_col, columns=columns_col, values=values_col) - 1

    # Calculate the t-score for the chosen confidence level
    t_scores = t.ppf((1 + confidence_level) / 2, degrees_freedom)

    # Calculate the margin of error for each group
    margin_of_error = t_scores * standard_errors

    # Calculate the confidence intervals
    lower_bound = acc - margin_of_error
    upper_bound = acc + margin_of_error

    return acc, standard_deviations, lower_bound, upper_bound

In [270]:
acc_model_prompt_mean, acc_model_prompt_std, acc_model_prompt_ci_lower, acc_model_prompt_ci_upper = calculate_confidence_intervals(df, "model", "prompt", "correct_answer", confidence_level=0.95)
acc_model_prompt_mean

/tmp/ipykernel_1153/3665219877.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  acc = df.groupby([columns_col, index_col]).mean().pivot_table(index=index_col, columns=columns_col, values=values_col)
/tmp/ipykernel_1153/3665219877.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sem is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  standard_errors = df.groupby([columns_col, index_col]).sem().pivot_table(index=index_col, columns=columns_col, values=values_col)
/tmp/ipykernel_1153/3665219877.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify n

prompt,None_None,None_kojima-01,None_zhou-01,qa-10_None,qa-12_None,qa-13_None,qa-16_None,qa-17_None,refl-01_None,zhou-01-ins_None
model,,,,,,,,,,
command-xlarge-nightly,0.47,0.45,0.55,0.53,0.53,0.58,0.55,0.52,0.51,0.56
flan-T5-xxl,0.62,0.62,0.58,0.62,0.65,0.62,0.59,0.61,0.58,0.59
gpt-3.5-turbo,0.75,0.76,0.75,0.73,0.74,0.73,0.73,0.71,0.72,0.75
gpt-4,0.81,0.86,0.89,0.84,0.86,0.84,0.84,0.81,0.84,0.85
text-davinci-002,0.59,0.56,0.65,0.55,0.61,0.61,0.65,0.53,0.57,0.54
text-davinci-003,0.61,0.61,0.66,0.63,0.65,0.63,0.60,0.62,0.64,0.65


In [271]:
acc_dataset_model_mean, acc_dataset_model_std, acc_dataset_model_ci_lower, acc_dataset_model_ci_upper = calculate_confidence_intervals(df, "dataset", "model", "correct_answer", confidence_level=0.95)
acc_dataset_model_mean

/tmp/ipykernel_1153/3665219877.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  acc = df.groupby([columns_col, index_col]).mean().pivot_table(index=index_col, columns=columns_col, values=values_col)
/tmp/ipykernel_1153/3665219877.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sem is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  standard_errors = df.groupby([columns_col, index_col]).sem().pivot_table(index=index_col, columns=columns_col, values=values_col)
/tmp/ipykernel_1153/3665219877.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify n

model,command-xlarge-nightly,flan-T5-xxl,gpt-3.5-turbo,gpt-4,text-davinci-002,text-davinci-003
dataset,,,,,,
commonsense_qa,0.66,0.85,0.76,0.85,0.75,0.75
med_qa,0.27,0.22,0.53,0.65,0.28,0.35
medmc_qa,0.31,0.35,0.63,0.80,0.41,0.41
open_book_qa,0.58,0.78,0.83,0.93,0.59,0.75
strategy_qa,0.57,0.62,0.73,0.85,0.63,0.63
worldtree,0.75,0.83,0.92,0.98,0.88,0.88


In [272]:
acc_model_prompt_mean, acc_model_prompt_std, acc_model_prompt_ci_lower, acc_model_prompt_ci_upper = calculate_confidence_intervals(df, "model", "prompt", "correct_answer", confidence_level=0.95)
acc_model_prompt_mean

/tmp/ipykernel_1153/3665219877.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  acc = df.groupby([columns_col, index_col]).mean().pivot_table(index=index_col, columns=columns_col, values=values_col)
/tmp/ipykernel_1153/3665219877.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sem is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  standard_errors = df.groupby([columns_col, index_col]).sem().pivot_table(index=index_col, columns=columns_col, values=values_col)
/tmp/ipykernel_1153/3665219877.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify n

prompt,None_None,None_kojima-01,None_zhou-01,qa-10_None,qa-12_None,qa-13_None,qa-16_None,qa-17_None,refl-01_None,zhou-01-ins_None
model,,,,,,,,,,
command-xlarge-nightly,0.47,0.45,0.55,0.53,0.53,0.58,0.55,0.52,0.51,0.56
flan-T5-xxl,0.62,0.62,0.58,0.62,0.65,0.62,0.59,0.61,0.58,0.59
gpt-3.5-turbo,0.75,0.76,0.75,0.73,0.74,0.73,0.73,0.71,0.72,0.75
gpt-4,0.81,0.86,0.89,0.84,0.86,0.84,0.84,0.81,0.84,0.85
text-davinci-002,0.59,0.56,0.65,0.55,0.61,0.61,0.65,0.53,0.57,0.54
text-davinci-003,0.61,0.61,0.66,0.63,0.65,0.63,0.60,0.62,0.64,0.65


In [273]:
acc_dataset_prompt_mean, acc_dataset_prompt_std, acc_dataset_prompt_ci_lower, acc_dataset_prompt_ci_upper = calculate_confidence_intervals(df, "dataset", "prompt", "correct_answer", confidence_level=0.95)
acc_dataset_prompt_mean

/tmp/ipykernel_1153/3665219877.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  acc = df.groupby([columns_col, index_col]).mean().pivot_table(index=index_col, columns=columns_col, values=values_col)
/tmp/ipykernel_1153/3665219877.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sem is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  standard_errors = df.groupby([columns_col, index_col]).sem().pivot_table(index=index_col, columns=columns_col, values=values_col)
/tmp/ipykernel_1153/3665219877.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify n

prompt,None_None,None_kojima-01,None_zhou-01,qa-10_None,qa-12_None,qa-13_None,qa-16_None,qa-17_None,refl-01_None,zhou-01-ins_None
dataset,,,,,,,,,,
commonsense_qa,0.74,0.75,0.78,0.78,0.78,0.80,0.75,0.71,0.79,0.79
med_qa,0.38,0.39,0.40,0.36,0.39,0.38,0.41,0.37,0.37,0.37
medmc_qa,0.46,0.44,0.54,0.48,0.52,0.49,0.53,0.51,0.44,0.45
open_book_qa,0.74,0.71,0.81,0.74,0.76,0.71,0.74,0.73,0.75,0.74
strategy_qa,0.64,0.73,0.67,0.66,0.66,0.72,0.68,0.66,0.62,0.66
worldtree,0.88,0.85,0.87,0.87,0.91,0.91,0.87,0.84,0.87,0.90


In [274]:
# Pandas does not work to compute the mean if there is None in the the column, I have no clue why
# It works when the column "correct_answer" is not defined as above. Also just works in pandas 1.5, not in 2.x
# acc_model_prompt = df.groupby(["model", "prompt"]).mean("correct_answer").pivot_table(index="model", columns="prompt", values="correct_answer")

# acc_model_prompt = df.groupby(["model", "prompt"]).mean().pivot_table(index="model", columns="prompt", values="correct_answer")
# acc_model_prompt

In [275]:
def calculate_confidence_intervals_simple_tables(dataframe, index_col, values_col, confidence_level=0.95):
    df = dataframe.copy()
    df = df.dropna(subset=['correct_answer'])
    df["correct_answer"] = df["correct_answer"].fillna(np.nan)
    from scipy.stats import t
    # Calculate mean accuracy scores
    acc = df.groupby([index_col]).mean()

    # Calculate standard error for each group
    standard_errors = df.groupby([index_col]).sem().pivot_table(index=index_col, values=values_col)

    # Calculate standard deviations for each group
    standard_deviations = df.groupby([index_col]).std().pivot_table(index=index_col, values=values_col)

    # Calculate degrees of freedom for each group
    degrees_freedom = df.groupby([index_col]).count().pivot_table(index=index_col, values=values_col) - 1

    # Calculate the t-score for the chosen confidence level
    t_scores = t.ppf((1 + confidence_level) / 2, degrees_freedom)

    # Calculate the margin of error for each group
    margin_of_error = t_scores * standard_errors

    # Calculate the confidence intervals
    lower_bound = acc - margin_of_error
    upper_bound = acc + margin_of_error

    acc["std"] = standard_deviations
    acc["ci_lower"] = lower_bound
    acc["ci_upper"] = upper_bound

    acc.rename(columns={values_col: "mean"}, inplace=True)

    acc.reset_index(inplace=True)

    return acc

In [276]:
acc_dataset = calculate_confidence_intervals_simple_tables(df, "dataset", "correct_answer", confidence_level=0.95)
acc_dataset

/tmp/ipykernel_1153/1312718999.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  acc = df.groupby([index_col]).mean()
/tmp/ipykernel_1153/1312718999.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sem is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  standard_errors = df.groupby([index_col]).sem().pivot_table(index=index_col, values=values_col)
/tmp/ipykernel_1153/1312718999.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  standard_deviations = df.groupby([index

,dataset,mean,std,ci_lower,ci_upper
0,commonsense_qa,0.77,0.42,0.75,0.79
1,med_qa,0.38,0.49,0.36,0.40
2,medmc_qa,0.49,0.50,0.46,0.51
3,open_book_qa,0.74,0.44,0.72,0.76
4,strategy_qa,0.67,0.47,0.65,0.69
5,worldtree,0.88,0.33,0.86,0.89


In [277]:
acc_model = calculate_confidence_intervals_simple_tables(df, "model", "correct_answer", confidence_level=0.95)
acc_model

/tmp/ipykernel_1153/1312718999.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  acc = df.groupby([index_col]).mean()
/tmp/ipykernel_1153/1312718999.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sem is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  standard_errors = df.groupby([index_col]).sem().pivot_table(index=index_col, values=values_col)
/tmp/ipykernel_1153/1312718999.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  standard_deviations = df.groupby([index

,model,mean,std,ci_lower,ci_upper
0,command-xlarge-nightly,0.52,0.50,0.50,0.55
1,flan-T5-xxl,0.61,0.49,0.59,0.63
2,gpt-3.5-turbo,0.74,0.44,0.72,0.76
3,gpt-4,0.85,0.36,0.83,0.86
4,text-davinci-002,0.59,0.49,0.56,0.61
5,text-davinci-003,0.63,0.48,0.61,0.65


In [278]:
acc_prompt = calculate_confidence_intervals_simple_tables(df, "prompt", "correct_answer", confidence_level=0.95)
acc_prompt

/tmp/ipykernel_1153/1312718999.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  acc = df.groupby([index_col]).mean()
/tmp/ipykernel_1153/1312718999.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sem is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  standard_errors = df.groupby([index_col]).sem().pivot_table(index=index_col, values=values_col)
/tmp/ipykernel_1153/1312718999.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  standard_deviations = df.groupby([index

,prompt,mean,std,ci_lower,ci_upper
0,None_None,0.64,0.48,0.61,0.67
1,None_kojima-01,0.64,0.48,0.62,0.67
2,None_zhou-01,0.68,0.47,0.65,0.70
3,qa-10_None,0.65,0.48,0.62,0.68
4,qa-12_None,0.67,0.47,0.64,0.70
5,qa-13_None,0.67,0.47,0.64,0.69
6,qa-16_None,0.66,0.47,0.63,0.69
7,qa-17_None,0.64,0.48,0.61,0.66
8,refl-01_None,0.64,0.48,0.61,0.67
9,zhou-01-ins_None,0.65,0.48,0.63,0.68


# Krippendorff's alpha

In [256]:
# KRIPPENDORFF ALPHA all datasets together, pooling over datasets
# Define bootstrapping parameters
n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility
random.seed(42)

# Perform bootstrapping and calculate means and confidence intervals
results = []

for (model, prompt), group in df.groupby(['model', 'prompt']):
    unique_datasets = group['dataset'].unique()
    dataset_means = {dataset: [] for dataset in unique_datasets}

    for _ in range(n_bootstraps):
        random_dataset = random.choice(unique_datasets)
        random_group = group[group['dataset'] == random_dataset]

        answer_label = random_group['answer_label'].sample(n=len(random_group), replace=True)
        indices = answer_label.index
        answer_from_choices = random_group.loc[indices, 'answer_from_choices']

        metric = krippendorff.alpha(reliability_data=[list(answer_label), list(answer_from_choices)], level_of_measurement='nominal')
        # there seem to be negative values, which are corrected to zero
        if metric < 0:
            metric = 0
        dataset_means[random_dataset].append(metric)

    all_means = np.concatenate(list(dataset_means.values()))
    mean = np.mean(all_means)

    # Calculate the pooled standard deviation
    dataset_stds = {dataset: np.std(means) for dataset, means in dataset_means.items()}

    squared_stds = [std ** 2 for std in dataset_stds.values()]
    pooled_std_dev = np.sqrt(sum(squared_stds) / len(squared_stds))

    # Calculate the margin of error using the z-score
    z_score = scipy.stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z_score * (pooled_std_dev / np.sqrt(len(unique_datasets)))

    # Compute the confidence interval
    ci_lower = mean - margin_of_error
    ci_upper = mean + margin_of_error

    results.append({
        'model': model,
        'prompt': prompt,
        'mean': mean,
        'std': pooled_std_dev,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper
    })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [257]:
kripp_model_prompt = results_df.copy()
kripp_model_prompt.head(1)

,model,prompt,mean,std,ci_lower,ci_upper
0,command-xlarge-nightly,None_None,0.26,0.09,0.18,0.33


In [258]:
kripp_model_prompt_mean = kripp_model_prompt.pivot_table(values='mean', index='model', columns='prompt')
kripp_model_prompt_std = kripp_model_prompt.pivot_table(values='std', index='model', columns='prompt')
kripp_model_prompt_ci_lower = kripp_model_prompt.pivot_table(values='ci_lower', index='model', columns='prompt')
kripp_model_prompt_ci_upper = kripp_model_prompt.pivot_table(values='ci_upper', index='model', columns='prompt')
kripp_model_prompt_mean

prompt,None_None,None_kojima-01,None_zhou-01,qa-10_None,qa-12_None,qa-13_None,qa-16_None,qa-17_None,refl-01_None,zhou-01-ins_None
model,,,,,,,,,,
command-xlarge-nightly,0.26,0.25,0.35,0.34,0.33,0.42,0.34,0.31,0.32,0.38
flan-T5-xxl,0.49,0.46,0.44,0.45,0.50,0.46,0.42,0.44,0.41,0.43
gpt-3.5-turbo,0.61,0.66,0.62,0.61,0.59,0.61,0.61,0.58,0.58,0.64
gpt-4,0.71,0.80,0.83,0.77,0.79,0.78,0.77,0.74,0.76,0.79
text-davinci-002,0.41,0.42,0.53,0.37,0.44,0.44,0.51,0.35,0.38,0.33
text-davinci-003,0.44,0.45,0.50,0.46,0.52,0.46,0.43,0.46,0.48,0.49


In [259]:
kripp_model_prompt_std

prompt,None_None,None_kojima-01,None_zhou-01,qa-10_None,qa-12_None,qa-13_None,qa-16_None,qa-17_None,refl-01_None,zhou-01-ins_None
model,,,,,,,,,,
command-xlarge-nightly,0.09,0.11,0.10,0.11,0.09,0.11,0.10,0.11,0.08,0.10
flan-T5-xxl,0.11,0.10,0.08,0.10,0.10,0.10,0.11,0.11,0.08,0.10
gpt-3.5-turbo,0.10,0.11,0.11,0.11,0.12,0.11,0.12,0.11,0.12,0.12
gpt-4,0.10,0.09,0.08,0.08,0.09,0.09,0.09,0.10,0.10,0.09
text-davinci-002,0.11,0.11,0.11,0.09,0.11,0.11,0.12,0.10,0.11,0.08
text-davinci-003,0.11,0.11,0.11,0.11,0.11,0.11,0.10,0.10,0.11,0.11


In [260]:
# acc_model_prompt/kripp_model_prompt.pivot_table(index="model", columns="prompt", values="mean")

In [267]:
# kripp_model_prompt_mean

In [266]:
# acc_model_prompt_mean

In [268]:
# should be correct, except in very similar scores, cause there can be small differences in the bootstrapping
# kripp_model_prompt_mean.rank(1) == acc_model_prompt_mean.rank(1)

In [249]:
# KRIPPENDORFF ALPHA dataset by prompt sum over models

# Define bootstrapping parameters
n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility
random.seed(42)

# Perform bootstrapping and calculate means and confidence intervals
results = []

for (dataset, prompt), group in df.groupby(['dataset', 'prompt']):
    bootstrapped_means = []
    for _ in range(n_bootstraps):
        answer_label = group['answer_label'].sample(n=len(group), replace=True)
        indices = answer_label.index
        answer_from_choices = group.loc[indices, 'answer_from_choices']

        metric = krippendorff.alpha(reliability_data=[list(answer_label), list(answer_from_choices)],level_of_measurement='nominal')
        if metric < 0:
            metric = 0
        bootstrapped_means.append(metric)
    
    mean = np.mean(bootstrapped_means)
    std = np.std(bootstrapped_means)
    lower = np.percentile(bootstrapped_means, (1 - confidence_level) / 2 * 100)
    upper = np.percentile(bootstrapped_means, (1 + confidence_level) / 2 * 100)
    
    results.append({
        'dataset': dataset,
        # 'model': model,
        'prompt': prompt,
        'mean': mean,
        'std': std,
        'ci_lower': lower,
        'ci_upper': upper
    })

# Convert the results to a DataFrame and pivot
results_df = pd.DataFrame(results)

In [250]:
kripp_dataset_prompt = results_df.copy()
kripp_dataset_prompt.head(1)

,dataset,prompt,mean,std,ci_lower,ci_upper
0,commonsense_qa,None_None,0.68,0.04,0.60,0.76


In [251]:
kripp_dataset_prompt_mean = kripp_dataset_prompt.pivot_table(values='mean', index='dataset', columns='prompt')
kripp_dataset_prompt_std = kripp_dataset_prompt.pivot_table(values='std', index='dataset', columns='prompt')
kripp_dataset_prompt_ci_lower = kripp_dataset_prompt.pivot_table(values='ci_lower', index='dataset', columns='prompt')
kripp_dataset_prompt_ci_upper = kripp_dataset_prompt.pivot_table(values='ci_upper', index='dataset', columns='prompt')
kripp_dataset_prompt_mean

prompt,None_None,None_kojima-01,None_zhou-01,qa-10_None,qa-12_None,qa-13_None,qa-16_None,qa-17_None,refl-01_None,zhou-01-ins_None
dataset,,,,,,,,,,
commonsense_qa,0.68,0.69,0.72,0.73,0.72,0.75,0.68,0.63,0.73,0.73
med_qa,0.21,0.22,0.23,0.19,0.22,0.21,0.25,0.20,0.19,0.19
medmc_qa,0.28,0.25,0.37,0.30,0.35,0.31,0.36,0.32,0.25,0.26
open_book_qa,0.65,0.61,0.74,0.65,0.67,0.61,0.64,0.63,0.66,0.65
strategy_qa,0.24,0.46,0.32,0.27,0.27,0.42,0.33,0.30,0.23,0.28
worldtree,0.84,0.79,0.83,0.82,0.88,0.87,0.82,0.78,0.82,0.86


In [252]:
# kripp_dataset_prompt_mean.std(1)

In [253]:
# kripp_dataset_prompt_mean.std(0)

In [254]:
# acc_dataset_prompt/kripp_dataset_prompt.pivot_table(values='mean', index='dataset', columns='prompt')

In [279]:
# kripp_dataset_prompt_mean

In [280]:
# acc_dataset_prompt_mean

In [281]:
# should be correct, except in very similar scores, cause there can be small differences in the bootstrapping
# kripp_dataset_prompt_mean.rank(1) == acc_dataset_prompt_mean.rank(1)

In [240]:
# KRIPPENDORFF ALPHA dataset by model, sum over prompts
# Define bootstrapping parameters
n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility
random.seed(42)

# Perform bootstrapping and calculate means and confidence intervals
results = []

for (dataset, model), group in df.groupby(['dataset', 'model']):
    bootstrapped_means = []
    for _ in range(n_bootstraps):
        answer_label = group['answer_label'].sample(n=len(group), replace=True)
        indices = answer_label.index
        answer_from_choices = group.loc[indices, 'answer_from_choices']

        metric = krippendorff.alpha(reliability_data=[list(answer_label), list(answer_from_choices)],level_of_measurement='nominal')
        if metric < 0:
            metric = 0
        bootstrapped_means.append(metric)
    
    mean = np.mean(bootstrapped_means)
    std = np.std(bootstrapped_means)
    lower = np.percentile(bootstrapped_means, (1 - confidence_level) / 2 * 100)
    upper = np.percentile(bootstrapped_means, (1 + confidence_level) / 2 * 100)
    
    results.append({
        'dataset': dataset,
        'model': model,
        # 'prompt': prompt,
        'std': std,
        'mean': mean,
        'ci_lower': lower,
        'ci_upper': upper
    })

# Convert the results to a DataFrame and pivot
results_df = pd.DataFrame(results)

In [241]:
kripp_dataset_model = results_df.copy()
kripp_dataset_model.head(1)

,dataset,model,std,mean,ci_lower,ci_upper
0,commonsense_qa,command-xlarge-nightly,0.03,0.57,0.50,0.64


In [242]:
kripp_dataset_model_mean = kripp_dataset_model.pivot_table(values='mean', index='dataset', columns='model')
kripp_dataset_model_std = kripp_dataset_model.pivot_table(values='std', index='dataset', columns='model')
kripp_dataset_model_ci_lower = kripp_dataset_model.pivot_table(values='ci_lower', index='dataset', columns='model')
kripp_dataset_model_ci_upper = kripp_dataset_model.pivot_table(values='ci_upper', index='dataset', columns='model')
kripp_dataset_model_mean

model,command-xlarge-nightly,flan-T5-xxl,gpt-3.5-turbo,gpt-4,text-davinci-002,text-davinci-003
dataset,,,,,,
commonsense_qa,0.57,0.81,0.70,0.82,0.68,0.68
med_qa,0.06,0.02,0.40,0.55,0.09,0.17
medmc_qa,0.08,0.10,0.51,0.73,0.20,0.21
open_book_qa,0.43,0.69,0.77,0.91,0.45,0.66
strategy_qa,0.10,0.23,0.44,0.69,0.20,0.22
worldtree,0.67,0.77,0.89,0.97,0.84,0.84


In [282]:
# acc_dataset_model_mean

In [283]:
# should be correct, except in very similar scores, cause there can be small differences in the bootstrapping
# kripp_dataset_model_mean.rank(1) == acc_dataset_model_mean.rank(1)

In [95]:
# acc_dataset_model/kripp_dataset_model.pivot_table(values='mean', index='dataset', columns='model')

In [284]:
# KRIPPENDORFF ALPHA only DATASET, sum over prompts and models
# Define bootstrapping parameters
n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility
random.seed(42)

# Perform bootstrapping and calculate means and confidence intervals
results = []

for dataset, group in df.groupby('dataset'):
    bootstrapped_means = []
    for _ in range(n_bootstraps):
        answer_label = group['answer_label'].sample(n=len(group), replace=True)
        indices = answer_label.index
        answer_from_choices = group.loc[indices, 'answer_from_choices']

        metric = krippendorff.alpha(reliability_data=[list(answer_label), list(answer_from_choices)],level_of_measurement='nominal')
        if metric < 0:
            metric = 0
        bootstrapped_means.append(metric)
    
    mean = np.mean(bootstrapped_means)
    std = np.std(bootstrapped_means)
    lower = np.percentile(bootstrapped_means, (1 - confidence_level) / 2 * 100)
    upper = np.percentile(bootstrapped_means, (1 + confidence_level) / 2 * 100)
    
    results.append({
        'dataset': dataset,
        # 'model': model,
        # 'prompt': prompt,
        'mean': mean,
        'std': std,
        'ci_lower': lower,
        'ci_upper': upper
    })

# Convert the results to a DataFrame and pivot
results_df = pd.DataFrame(results)

In [285]:
kripp_dataset = results_df.copy()

In [286]:
kripp_dataset

,dataset,mean,std,ci_lower,ci_upper
0,commonsense_qa,0.71,0.01,0.68,0.73
1,med_qa,0.21,0.01,0.19,0.24
2,medmc_qa,0.31,0.02,0.28,0.34
3,open_book_qa,0.65,0.01,0.63,0.68
4,strategy_qa,0.31,0.02,0.27,0.36
5,worldtree,0.83,0.01,0.81,0.85


In [287]:
# KRIPPENDORFF ALPHA model scores by summing over prompts and datasets
# BOOTSTRAPPING SEPERATELY FOR EACH DATASET, SINCE THE NUMBER OF CHOICES IS DIFFERENT
import numpy as np
import pandas as pd
import random
import scipy.stats

n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility
random.seed(42)

# Perform bootstrapping and calculate means and confidence intervals
results = []

for model, group in df.groupby('model'):
    unique_datasets = group['dataset'].unique()
    dataset_means = {dataset: [] for dataset in unique_datasets}

    for _ in range(n_bootstraps):

        random_dataset = random.choice(unique_datasets)
        random_group = group[group['dataset'] == random_dataset]

        answer_label = random_group['answer_label'].sample(n=len(random_group), replace=True)
        indices = answer_label.index
        answer_from_choices = random_group.loc[indices, 'answer_from_choices']

        metric = krippendorff.alpha(reliability_data=[list(answer_label), list(answer_from_choices)], level_of_measurement='nominal')
        if metric < 0:
            metric = 0
        dataset_means[random_dataset].append(metric)

    all_means = np.concatenate(list(dataset_means.values()))
    mean = np.mean(all_means)

    # Calculate the pooled standard deviation
    dataset_stds = {dataset: np.std(means) for dataset, means in dataset_means.items()}

    squared_stds = [std ** 2 for std in dataset_stds.values()]
    pooled_std_dev = np.sqrt(sum(squared_stds) / len(squared_stds))

    # Calculate the margin of error using the z-score
    z_score = scipy.stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z_score * (pooled_std_dev / np.sqrt(len(unique_datasets)))

    # Compute the confidence interval
    ci_lower = mean - margin_of_error
    ci_upper = mean + margin_of_error

    results.append({
        'model': model,
        'mean': mean,
        'std': pooled_std_dev,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper
    })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [288]:
kripp_model = results_df.copy()

In [289]:
kripp_model

,model,mean,std,ci_lower,ci_upper
0,command-xlarge-nightly,0.32,0.04,0.29,0.35
1,flan-T5-xxl,0.45,0.03,0.42,0.47
2,gpt-3.5-turbo,0.62,0.04,0.59,0.65
3,gpt-4,0.78,0.03,0.76,0.81
4,text-davinci-002,0.41,0.04,0.38,0.44
5,text-davinci-003,0.47,0.04,0.45,0.50


In [290]:
# KRIPPENDORFF ALPHA prompt scores by summing over models and datasets
# BOOTSTRAPPING SEPERATELY FOR EACH DATASET, SINCE THE NUMBER OF CHOICES IS DIFFERENT
import numpy as np
import pandas as pd
import random
import scipy.stats

n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility
random.seed(42)

# Perform bootstrapping and calculate means and confidence intervals
results = []

for prompt, group in df.groupby('prompt'):
    unique_datasets = group['dataset'].unique()
    dataset_means = {dataset: [] for dataset in unique_datasets}

    for _ in range(n_bootstraps):

        random_dataset = random.choice(unique_datasets)
        random_group = group[group['dataset'] == random_dataset]

        answer_label = random_group['answer_label'].sample(n=len(random_group), replace=True)
        indices = answer_label.index
        answer_from_choices = random_group.loc[indices, 'answer_from_choices']

        metric = krippendorff.alpha(reliability_data=[list(answer_label), list(answer_from_choices)], level_of_measurement='nominal')
        if metric < 0:
            metric = 0
        dataset_means[random_dataset].append(metric)

    all_means = np.concatenate(list(dataset_means.values()))
    mean = np.mean(all_means)

    # Calculate the pooled standard deviation
    dataset_stds = {dataset: np.std(means) for dataset, means in dataset_means.items()}

    squared_stds = [std ** 2 for std in dataset_stds.values()]
    pooled_std_dev = np.sqrt(sum(squared_stds) / len(squared_stds))

    # Calculate the margin of error using the z-score
    z_score = scipy.stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z_score * (pooled_std_dev / np.sqrt(len(unique_datasets)))

    # Compute the confidence interval
    ci_lower = mean - margin_of_error
    ci_upper = mean + margin_of_error

    results.append({
        'prompt': prompt,
        'mean': mean,
        'std': pooled_std_dev,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper
    })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [291]:
kripp_prompt = results_df.copy()
kripp_prompt.to_csv('krippendorff_alpha_prompt.csv', index=False)

In [292]:
kripp_prompt

,prompt,mean,std,ci_lower,ci_upper
0,None_None,0.49,0.05,0.45,0.52
1,None_kojima-01,0.51,0.05,0.47,0.55
2,None_zhou-01,0.53,0.05,0.50,0.57
3,qa-10_None,0.50,0.05,0.46,0.54
4,qa-12_None,0.52,0.05,0.48,0.56
5,qa-13_None,0.54,0.05,0.51,0.58
6,qa-16_None,0.51,0.05,0.47,0.55
7,qa-17_None,0.47,0.05,0.43,0.51
8,refl-01_None,0.49,0.05,0.45,0.53
9,zhou-01-ins_None,0.50,0.05,0.46,0.54


### creating tables

In [293]:
# changing the naming of prompts, models and datasets

prompt_replacements = {
    'None_None': 'Direct',
    'qa-10_None': 'Plan',
    'qa-12_None': 'Articulate',
    'qa-13_None': 'Rephrase',
    'qa-16_None': 'Elaborate',
    'qa-17_None': 'Converse',
    'zhou-01-ins_None': 'Zhou-instruction',
    'refl-01_None': 'Self-critique',
    'None_kojima-01': 'Kojima',
    'None_zhou-01': 'Zhou'
}

In [294]:
# changing the naming of prompts, models and datasets

model_replacements = {
    'command-xlarge-nightly': 'Command-XL',
    'flan-T5-xxl': 'Flan-T5-XXL',
    'gpt-3.5-turbo': 'GPT-3.5-turbo',
    'gpt-4': 'GPT-4',
    'text-davinci-002': 'Davinci-002',
    'text-davinci-003': 'Davinci-003',
}

In [295]:
# changing the naming of prompts, models and datasets

dataset_replacements = {
    'commonsense_qa': 'CommonsenseQA',
    'med_qa': 'MedQA',
    'medmc_qa': 'MedMCQA',
    'open_book_qa': 'OpenBookQA',
    'strategy_qa': 'StrategyQA',
    'worldtree': 'WorldTree v2',
}

In [296]:
def format_decimal(value):
    if isinstance(value, float):
        formatted_value = f'{value:.2f}'.lstrip('0')
    else:
        formatted_value = value
    return formatted_value

In [331]:
# df = kripp_prompt.copy()
# df = kripp_model.copy()
# df = kripp_dataset.copy()
# df = kripp_model_prompt[kripp_model_prompt.model=='gpt-4'].copy()
# df = kripp_dataset_model.copy()
# df = acc_prompt.copy()
# df = acc_dataset.copy()
df = acc_model.copy()


In [332]:
df

,model,mean,std,ci_lower,ci_upper
0,command-xlarge-nightly,0.52,0.50,0.50,0.55
1,flan-T5-xxl,0.61,0.49,0.59,0.63
2,gpt-3.5-turbo,0.74,0.44,0.72,0.76
3,gpt-4,0.85,0.36,0.83,0.86
4,text-davinci-002,0.59,0.49,0.56,0.61
5,text-davinci-003,0.63,0.48,0.61,0.65


In [333]:
df.drop(columns="std", inplace=True)

In [334]:
if 'prompt' in df.columns:
    df['prompt'] = df['prompt'].map(prompt_replacements)
if 'model' in df.columns:
    df['model'] = df['model'].map(model_replacements)
if 'dataset' in df.columns:
    df['dataset'] = df['dataset'].map(dataset_replacements)

In [335]:
df.rename(columns={col: col.capitalize() for col in ['prompt', 'model', 'dataset']}, inplace=True)
if 'mean' in df.columns:
    df.sort_values(by='mean', inplace=True, ascending=False)
    df.reset_index(drop=True, inplace=True)

In [336]:
# if isinstance(df.index, pd.RangeIndex) or pd.api.types.is_integer_dtype(df.index):
#     df.set_index(df.columns[0], inplace=True)

In [337]:
df = df.round(2)
df = df.applymap(format_decimal)
df

,Model,mean,ci_lower,ci_upper
0,GPT-4,.85,.83,.86
1,GPT-3.5-turbo,.74,.72,.76
2,Davinci-003,.63,.61,.65
3,Flan-T5-XXL,.61,.59,.63
4,Davinci-002,.59,.56,.61
5,Command-XL,.52,.50,.55


In [338]:
# df["Krippendorff's \\alpha"] = df['mean'] + " ({\\scriptsize" + df['ci_lower'] + ", " + df['ci_upper'] + "})"

# # Drop the original 'mean', 'ci_lower', and 'ci_upper' columns
# df = df.drop(columns=['mean', 'ci_lower', 'ci_upper'])

# # Convert the DataFrame to a LaTeX table
# latex_table = df.to_latex(column_format="l" + "c" * len(df.columns), float_format="{:.2f}".format, escape=False, index=False)

# print(latex_table)

In [339]:
# df["Krippendorff's \\alpha"] = df['mean'] + " ({\\scriptsize" + df['ci_lower'] + ", " + df['ci_upper'] + "})"
df["Accuracy"] = df['mean'] + " ({\\scriptsize" + df['ci_lower'] + ", " + df['ci_upper'] + "})"

# Drop the original 'mean', 'ci_lower', and 'ci_upper' columns
df = df.drop(columns=['mean', 'ci_lower', 'ci_upper'])

# Create the table from the DataFrame
latex_table = df.to_latex(column_format="l" + "c" * len(df.columns), float_format="{:.2f}".format, escape=False, index=False)

# Add horizontal lines and bold text for the header row
latex_table = latex_table.replace("\\toprule", "\\hline")
latex_table = latex_table.replace("\\midrule", "\\hline")
latex_table = latex_table.replace("\\bottomrule", "\\hline")
header_row = "&".join([f"\\textbf{{{col}}}" for col in df.columns])
latex_table = latex_table.replace("&".join(df.columns), header_row)

# Wrap the table in a table environment
latex_table = f"""\
\\begin{{table}}
\\centering
{latex_table}
\\caption{{Your caption here.}}
\\label{{tab:your_label}}
\\end{{table}}
"""

print(latex_table)

\begin{table}
\centering
\begin{tabular}{lcc}
\hline
        Model &                    Accuracy \\
\hline
        GPT-4 & .85 ({\scriptsize.83, .86}) \\
GPT-3.5-turbo & .74 ({\scriptsize.72, .76}) \\
  Davinci-003 & .63 ({\scriptsize.61, .65}) \\
  Flan-T5-XXL & .61 ({\scriptsize.59, .63}) \\
  Davinci-002 & .59 ({\scriptsize.56, .61}) \\
   Command-XL & .52 ({\scriptsize.50, .55}) \\
\hline
\end{tabular}

\caption{Your caption here.}
\label{tab:your_label}
\end{table}



/tmp/ipykernel_1153/897540160.py:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(column_format="l" + "c" * len(df.columns), float_format="{:.2f}".format, escape=False, index=False)


In [192]:
mean_pivot = kripp_dataset_model_mean.copy().round(2).clip(0,1).applymap(format_decimal)
ci_lower = kripp_dataset_model_ci_lower.copy().round(2).clip(0,1).applymap(format_decimal)
ci_upper = kripp_dataset_model_ci_upper.copy().round(2).clip(0,1).applymap(format_decimal)

for df in [mean_pivot, ci_lower, ci_upper]:
    df.index = df.index.map(dataset_replacements)
    df.columns = df.columns.map(model_replacements)

In [340]:
mean_pivot = acc_dataset_model_mean.copy().round(2).clip(0,1).applymap(format_decimal)
ci_lower = acc_dataset_model_ci_lower.copy().round(2).clip(0,1).applymap(format_decimal)
ci_upper = acc_dataset_model_ci_upper.copy().round(2).clip(0,1).applymap(format_decimal)

for df in [mean_pivot, ci_lower, ci_upper]:
    df.index = df.index.map(dataset_replacements)
    df.columns = df.columns.map(model_replacements)

In [221]:
mean_pivot = kripp_dataset_prompt_mean.copy().round(2).clip(0,1).applymap(format_decimal)
ci_lower = kripp_dataset_prompt_ci_lower.copy().round(2).clip(0,1).applymap(format_decimal)
ci_upper = kripp_dataset_prompt_ci_upper.copy().round(2).clip(0,1).applymap(format_decimal)

for df in [mean_pivot, ci_lower, ci_upper]:
    df.index = df.index.map(dataset_replacements)
    df.columns = df.columns.map(prompt_replacements)

mean_pivot = mean_pivot.T
ci_lower = ci_lower.T
ci_upper = ci_upper.T

In [343]:
mean_pivot = acc_dataset_prompt_mean.copy().round(2).clip(0,1).applymap(format_decimal)
ci_lower = acc_dataset_prompt_ci_lower.copy().round(2).clip(0,1).applymap(format_decimal)
ci_upper = acc_dataset_prompt_ci_upper.copy().round(2).clip(0,1).applymap(format_decimal)

for df in [mean_pivot, ci_lower, ci_upper]:
    df.index = df.index.map(dataset_replacements)
    df.columns = df.columns.map(prompt_replacements)

mean_pivot = mean_pivot.T
ci_lower = ci_lower.T
ci_upper = ci_upper.T

In [218]:
mean_pivot = kripp_model_prompt_mean.copy().round(2).clip(0,1).applymap(format_decimal)
ci_lower = kripp_model_prompt_ci_lower.copy().round(2).clip(0,1).applymap(format_decimal)
ci_upper = kripp_model_prompt_ci_upper.copy().round(2).clip(0,1).applymap(format_decimal)

for df in [mean_pivot, ci_lower, ci_upper]:
    df.index = df.index.map(model_replacements)
    df.columns = df.columns.map(prompt_replacements)

mean_pivot = mean_pivot.T
ci_lower = ci_lower.T
ci_upper = ci_upper.T

In [346]:
mean_pivot = acc_model_prompt_mean.copy().round(2).clip(0,1).applymap(format_decimal)
ci_lower = acc_model_prompt_ci_lower.copy().round(2).clip(0,1).applymap(format_decimal)
ci_upper = acc_model_prompt_ci_upper.copy().round(2).clip(0,1).applymap(format_decimal)

for df in [mean_pivot, ci_lower, ci_upper]:
    df.index = df.index.map(model_replacements)
    df.columns = df.columns.map(prompt_replacements)

mean_pivot = mean_pivot.T
ci_lower = ci_lower.T
ci_upper = ci_upper.T

In [347]:
mean_pivot

model,Command-XL,Flan-T5-XXL,GPT-3.5-turbo,GPT-4,Davinci-002,Davinci-003
prompt,,,,,,
Direct,.47,.62,.75,.81,.59,.61
Kojima,.45,.62,.76,.86,.56,.61
Zhou,.55,.58,.75,.89,.65,.66
Plan,.53,.62,.73,.84,.55,.63
Articulate,.53,.65,.74,.86,.61,.65
Rephrase,.58,.62,.73,.84,.61,.63
Elaborate,.55,.59,.73,.84,.65,.60
Converse,.52,.61,.71,.81,.53,.62
Self-critique,.51,.58,.72,.84,.57,.64


In [349]:
# Combine the DataFrames

final_df = mean_pivot.copy()
for col in mean_pivot.columns:
    final_df[col] = mean_pivot[col] + " ({\\scriptsize " + ci_lower[col] + ", " + ci_upper[col] + "})"

# Apply custom formatting (use LaTeX format for a scientific paper)
latex_table = final_df.to_latex(column_format="l" + "c" * len(final_df.columns), float_format="{:.2f}".format, escape=False)

print(latex_table)


\begin{tabular}{lcccccc}
\toprule
model &                    Command-XL &                   Flan-T5-XXL &                 GPT-3.5-turbo &                         GPT-4 &                   Davinci-002 &                   Davinci-003 \\
prompt           &                               &                               &                               &                               &                               &                               \\
\midrule
Direct           &  .47 ({\scriptsize .40, .54}) &  .62 ({\scriptsize .55, .69}) &  .75 ({\scriptsize .69, .81}) &  .81 ({\scriptsize .76, .87}) &  .59 ({\scriptsize .52, .66}) &  .61 ({\scriptsize .54, .68}) \\
Kojima           &  .45 ({\scriptsize .38, .52}) &  .62 ({\scriptsize .55, .69}) &  .76 ({\scriptsize .70, .82}) &  .86 ({\scriptsize .81, .91}) &  .56 ({\scriptsize .49, .63}) &  .61 ({\scriptsize .54, .68}) \\
Zhou             &  .55 ({\scriptsize .48, .62}) &  .58 ({\scriptsize .51, .65}) &  .75 ({\scriptsize .69, .81}) &  .89 

/tmp/ipykernel_1153/4069475834.py:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = final_df.to_latex(column_format="l" + "c" * len(final_df.columns), float_format="{:.2f}".format, escape=False)


###  word

In [ ]:
# from docx import Document
# doc = Document()

# df["Krippendorff's " + chr(945)] = df['mean'].round(2).astype(str) + " (" + df['ci_lower'].round(2).astype(str) + ", " + df['ci_upper'].round(2).astype(str) + ")"

# # Drop the original 'mean', 'ci_lower', and 'ci_upper' columns
# df = df.drop(columns=['mean', 'ci_lower', 'ci_upper'])

# # Add a table to the document
# table = doc.add_table(rows=df.shape[0] + 1, cols=df.shape[1])

# # Populate the table with the DataFrame content
# for i, col in enumerate(df.columns):
#     table.cell(0, i).text = col

# for i, row in df.iterrows():
#     for j, value in enumerate(row):
#         table.cell(i + 1, j).text = str(value)

# # Save the Word document
# name = "dataframe_" + "_".join(df.columns) + ".docx"
# doc.save(name)

### test runs for checking scores

In [449]:
# test run to see if the pooling is correct for the dataset option
# bootstrapping per dataset is not needed for the dataset option, we can check if it leads to the same results

import numpy as np
import pandas as pd
import random
import scipy.stats

n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility
random.seed(42)

# Perform bootstrapping and calculate means and confidence intervals
results = []

for (dataset), group in df.groupby(['dataset']):
    unique_datasets = group['model'].unique()
    dataset_means = {dataset: [] for dataset in unique_datasets}

    for _ in range(n_bootstraps):

        random_dataset = random.choice(unique_datasets)
        random_group = group[group['model'] == random_dataset]

        answer_label = random_group['answer_label'].sample(n=len(random_group), replace=True)
        indices = answer_label.index
        answer_from_choices = random_group.loc[indices, 'answer_from_choices']

        metric = krippendorff.alpha(reliability_data=[list(answer_label), list(answer_from_choices)], level_of_measurement='nominal')
        if metric < 0:
            metric = 0
        dataset_means[random_dataset].append(metric)

    all_means = np.concatenate(list(dataset_means.values()))
    mean = np.mean(all_means)

    # Calculate the pooled standard deviation
    dataset_stds = {dataset: np.std(means) for dataset, means in dataset_means.items()}

    squared_stds = [std ** 2 for std in dataset_stds.values()]
    pooled_std_dev = np.sqrt(sum(squared_stds) / len(squared_stds))

    # Calculate the margin of error using the z-score
    z_score = scipy.stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z_score * (pooled_std_dev / np.sqrt(len(unique_datasets)))

    # Compute the confidence interval
    ci_lower = mean - margin_of_error
    ci_upper = mean + margin_of_error

    results.append({
        'dataset': dataset,
        'mean': mean,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper
    })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

/tmp/ipykernel_273/1604803771.py:16: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for (dataset), group in df.groupby(['dataset']):


In [450]:
# test if the pooling is correct for the dataset option
results_df

,dataset,mean,ci_lower,ci_upper
0,commonsense_qa,0.7111,0.6876,0.7346
1,med_qa,0.2207,0.1956,0.2459
2,medmc_qa,0.3022,0.2737,0.3306
3,open_book_qa,0.6512,0.6248,0.6777
4,strategy_qa,0.3085,0.2605,0.3565
5,worldtree,0.8329,0.8139,0.8519


In [451]:
# should be very similar to
kripp_dataset

,dataset,mean,ci_lower,ci_upper
0,commonsense_qa,0.7073,0.6819,0.7314
1,med_qa,0.2184,0.1920,0.2456
2,medmc_qa,0.3051,0.2749,0.3358
3,open_book_qa,0.6516,0.6258,0.6780
4,strategy_qa,0.3436,0.2961,0.3935
5,worldtree,0.8325,0.8125,0.8525


these two should roughly be the same

In [461]:
kripp_dataset_model.pivot_table(values='mean', index='dataset', columns='model').mean()

model
command-xlarge-nightly   0.3103
flan-T5-xxl              0.4359
gpt-3.5-turbo            0.6212
gpt-4                    0.7827
text-davinci-002         0.4138
text-davinci-003         0.4648
dtype: float64

In [462]:
kripp_model[['model','mean']]

,model,mean
0,command-xlarge-nightly,0.3135
1,flan-T5-xxl,0.4325
2,gpt-3.5-turbo,0.6334
3,gpt-4,0.7835
4,text-davinci-002,0.4185
5,text-davinci-003,0.4698


In [ ]:
# # INCORRECT SINCE SUMMING OVER DATASETS
# # RESULTS WILL BE TOO HIGH

# # KRIPPENDORFF ALPHA all datasets together
# # Define bootstrapping parameters
# n_bootstraps = 1000
# confidence_level = 0.95
# np.random.seed(42)  # Set the seed for reproducibility
# random.seed(42)

# # Perform bootstrapping and calculate means and confidence intervals
# results = []

# for (model, prompt), group in df.groupby(['model', 'prompt']):
#     bootstrapped_means = []
#     for _ in range(n_bootstraps):
#         answer_label = group['answer_label'].sample(n=len(group), replace=True)
#         indices = answer_label.index
#         answer_from_choices = group.loc[indices, 'answer_from_choices']

#         metric = krippendorff.alpha(reliability_data=[list(answer_label), list(answer_from_choices)],level_of_measurement='nominal')
        # if metric < 0:
        #     metric = 0
#         bootstrapped_means.append(metric)
    
#     mean = np.mean(bootstrapped_means)
#     lower = np.percentile(bootstrapped_means, (1 - confidence_level) / 2 * 100)
#     upper = np.percentile(bootstrapped_means, (1 + confidence_level) / 2 * 100)
    
#     results.append({
#         # 'dataset': dataset,
#         'model': model,
#         'prompt': prompt,
#         'mean': mean,
#         'ci_lower': lower,
#         'ci_upper': upper
#     })

# # Convert the results to a DataFrame and pivot
# results_df = pd.DataFrame(results)

# results_df.pivot_table(values='mean', index='model', columns='prompt')

In [ ]:
# # Define bootstrapping parameters
# n_bootstraps = 1000
# confidence_level = 0.95
# np.random.seed(42)  # Set the seed for reproducibility
# random.seed(42)

# # Perform bootstrapping and calculate means and confidence intervals
# results = []

# for (model, prompt, dataset), group in df.groupby(['model', 'prompt', 'dataset']):
#     bootstrapped_means = []
#     for _ in range(n_bootstraps):
#         resampled = group['correct_answer'].sample(n=len(group), replace=True)
#         bootstrapped_means.append(resampled.mean())
    
#     mean = np.mean(bootstrapped_means)
#     lower = np.percentile(bootstrapped_means, (1 - confidence_level) / 2 * 100)
#     upper = np.percentile(bootstrapped_means, (1 + confidence_level) / 2 * 100)
    
#     results.append({
#         'dataset': dataset,
#         'model': model,
#         'prompt': prompt,
#         'mean': mean,
#         'ci_lower': lower,
#         'ci_upper': upper
#     })

# # Convert the results to a DataFrame and pivot
# results_df = pd.DataFrame(results)
# mean_pivot = results_df.pivot_table(values='mean', index='model', columns='prompt')
# ci_pivot_lower = results_df.pivot_table(values='ci_lower', index='model', columns='prompt')
# ci_pivot_upper = results_df.pivot_table(values='ci_upper', index='model', columns='prompt')

In [ ]:
# # ACCURACY BOOTSRTAPPING
# # Define bootstrapping parameters
# n_bootstraps = 1000
# confidence_level = 0.95
# np.random.seed(42)  # Set the seed for reproducibility
# random.seed(42)

# # Perform bootstrapping and calculate means and confidence intervals
# results = []

# for (model, prompt), group in df.groupby(['model', 'prompt']):
#     bootstrapped_means = []
#     for _ in range(n_bootstraps):
#         resampled = group['correct_answer'].sample(n=len(group), replace=True)
#         bootstrapped_means.append(resampled.mean())
    
#     mean = np.mean(bootstrapped_means)
#     lower = np.percentile(bootstrapped_means, (1 - confidence_level) / 2 * 100)
#     upper = np.percentile(bootstrapped_means, (1 + confidence_level) / 2 * 100)
    
#     results.append({
#         'model': model,
#         'prompt': prompt,
#         'mean': mean,
#         'ci_lower': lower,
#         'ci_upper': upper
#     })

# # Convert the results to a DataFrame and pivot
# results_df = pd.DataFrame(results)

### inspection of the dataframe

In [481]:
filtered_df = df[(df.dataset == "strategy_qa")& (df.model == "command-xlarge-nightly") & (df.prompt == "None_zhou-01")]
# filtered_df.head(1)

In [ ]:
# filtered_df[['answer', 'answer_label']]

In [ ]:
# filtered_df["correct_answer"].unique(), filtered_df["answer_from_choices"].unique()

In [ ]:
# filtered_df["correct_answer"].value_counts(), filtered_df["answer_from_choices"].value_counts()

In [ ]:
# filtered_df["correct_answer"].count(), filtered_df["answer_from_choices"].count()

In [ ]:
krippendorff.alpha(reliability_data=[list(filtered_df["answer_label"]), list(filtered_df["answer_from_choices"].replace("", np.nan))],level_of_measurement='nominal')

0.0493

In [ ]:
# filtered_df[["answer","answer_from_choices"]][:200]

In [ ]:
# df.groupby(['dataset'])['number_choices'].unique()

### functions extra, non pandas

In [29]:
def table_function_new(eval:dict):
    import pandas as pd
    eval_dict = pd.json_normalize(eval).to_dict('records')[0]
    eval_list = list(eval_dict.keys())
    datasets = sorted(list(eval.keys()))

    models = []
    prompts = []
    for i in eval_list:
        # fast fix for chat gpt model:
        i = i.replace("gpt-3.5-turbo","gpt-3-5-turbo")
        dataset,split,metric,model,prompt = i.split(".")
        model = model.replace("gpt-3-5-turbo","gpt-3.5-turbo")
        if model not in models:
            models.append(model)
        if prompt not in prompts:
            prompts.append(prompt)
            
    models = sorted(models)

    if "None_None_None" in prompts: prompts.remove("None_None_None")

    # no instructions implemented yet
    # instructions = []
    cot_triggers = []
    for i in prompts:
        instruction, cot_trigger, _ = i.split("_")
        # if instruction not in instructions:
        #     instructions.append(instruction)

        # old: only cot_trigger
        # if cot_trigger not in cot_triggers:
        #     cot_triggers.append(cot_trigger)

        # new: instruction + cot_trigger
        if cot_trigger not in cot_triggers:
            cot_triggers.append(instruction + "_" + cot_trigger)

    cot_triggers = sorted(cot_triggers)

    cot_trigger_header = sorted(cot_triggers*len(models))
    model_header = models*len(cot_triggers)

    # Create a dictionary to store the data
    data_dict = {}
    for k,v in eval_dict.items():
        # fast fix for chat gpt model:
        k = k.replace("gpt-3.5-turbo","gpt-3-5-turbo")
        dataset,split,metric,model,prompt = k.split(".")
        model = model.replace("gpt-3-5-turbo","gpt-3.5-turbo")
        instruction, cot_trigger, _ = prompt.split("_")
        # old: only cot_trigger
        # df.loc[dataset, (cot_trigger, model)] = v

        # new: instruction + cot_trigger
        col_name = instruction + "_" + cot_trigger + "_" + model
        if dataset not in data_dict:
            data_dict[dataset] = {}
        data_dict[dataset][col_name] = v

    # Create a dataframe from the dictionary
    df = pd.DataFrame.from_dict(data_dict, orient='index')

    # Calculate the average row and add it to the dataframe
    df.loc['Average'] = df.mean()

    # Round the dataframe to 2 decimal places
    df = df.round(2)

    return df

In [41]:
df = table_function_new(eval)
df

,None_None_command-xlarge-nightly,None_kojima-01_command-xlarge-nightly,None_zhou-01_command-xlarge-nightly,qa-10_None_command-xlarge-nightly,qa-12_None_command-xlarge-nightly,qa-13_None_command-xlarge-nightly,qa-16_None_command-xlarge-nightly,qa-17_None_command-xlarge-nightly,refl-01_None_command-xlarge-nightly,zhou-01-ins_None_command-xlarge-nightly,...,None_None_text-davinci-003,None_kojima-01_text-davinci-003,None_zhou-01_text-davinci-003,qa-10_None_text-davinci-003,qa-12_None_text-davinci-003,qa-13_None_text-davinci-003,qa-16_None_text-davinci-003,qa-17_None_text-davinci-003,refl-01_None_text-davinci-003,zhou-01-ins_None_text-davinci-003
commonsense_qa,0.52,0.48,0.73,0.70,0.70,0.76,0.67,0.61,0.70,0.73,...,0.73,0.67,0.67,0.70,0.79,0.67,0.58,0.64,0.76,0.73
med_qa,0.18,0.30,0.27,0.27,0.27,0.24,0.33,0.30,0.21,0.24,...,0.30,0.33,0.39,0.36,0.30,0.39,0.36,0.24,0.33,0.36
medmc_qa,0.27,0.15,0.42,0.27,0.30,0.36,0.36,0.36,0.24,0.33,...,0.36,0.36,0.48,0.45,0.36,0.36,0.39,0.42,0.39,0.33
open_book_qa,0.58,0.42,0.61,0.58,0.58,0.58,0.55,0.61,0.61,0.64,...,0.64,0.58,0.79,0.73,0.79,0.61,0.64,0.73,0.64,0.82
strategy_qa,0.55,0.58,0.24,0.12,0.09,0.12,0.33,0.30,0.06,0.09,...,0.61,0.64,0.64,0.58,0.67,0.70,0.55,0.58,0.64,0.64
worldtree,0.61,0.61,0.76,0.73,0.82,0.85,0.85,0.64,0.79,0.82,...,0.94,0.91,0.82,0.82,0.91,0.85,0.82,0.91,0.91,0.88
Average,0.45,0.42,0.51,0.44,0.46,0.48,0.52,0.47,0.43,0.47,...,0.60,0.58,0.63,0.61,0.64,0.60,0.56,0.59,0.61,0.63


In [49]:
df = (df[df.index=="Average"])
#df.to_csv("Average_values_df.csv")
df

,None_None_command-xlarge-nightly,None_kojima-01_command-xlarge-nightly,None_zhou-01_command-xlarge-nightly,qa-10_None_command-xlarge-nightly,qa-12_None_command-xlarge-nightly,qa-13_None_command-xlarge-nightly,qa-16_None_command-xlarge-nightly,qa-17_None_command-xlarge-nightly,refl-01_None_command-xlarge-nightly,zhou-01-ins_None_command-xlarge-nightly,...,None_None_text-davinci-003,None_kojima-01_text-davinci-003,None_zhou-01_text-davinci-003,qa-10_None_text-davinci-003,qa-12_None_text-davinci-003,qa-13_None_text-davinci-003,qa-16_None_text-davinci-003,qa-17_None_text-davinci-003,refl-01_None_text-davinci-003,zhou-01-ins_None_text-davinci-003
Average,0.45,0.42,0.51,0.44,0.46,0.48,0.52,0.47,0.43,0.47,...,0.6,0.58,0.63,0.61,0.64,0.6,0.56,0.59,0.61,0.63


In [12]:
import json, re
import pandas as pd
def clean_column(df, col_name):
    pattern = r"^.?'(.?)'.?'(.?)'.?'(.?)'.?'(.?)'.*$"
    return(df[col_name].apply(lambda x: re.sub(pattern, r"'\3", x)[1:]))

def json_to_dataframe_3(json_data):
    df_data = []
    for category, data in json_data.items():
        for subset, questions in data.items():
            for question in questions:
                for generated_cot in question['generated_cot']:
                    row = {
                        'dataset': category,
                        'split': subset,
                        'id': question['id'],
                        'model': generated_cot['model'],
                        'instruction': generated_cot['instruction'],
                        'cot_trigger': generated_cot["cot_trigger"],
                        'generated_cot': generated_cot['cot'],
                        'correct_answer': generated_cot['answers'][0]['correct_answer'],
                    }
                    df_data.append(row)
    df = pd.DataFrame(df_data)
    # df["model"] = clean_column(df, "model")
    return df


In [13]:
df["model"][0]

"'name': 'command-xlarge-nightly', 'temperature': 0, 'max_tokens': 512}"

In [10]:
path = "/home/kon/work/ThoughtSource/libs/cot/cot/datasets/thoughtsource/thoughtsource_33_paper.json"


with open(path, "r") as read_file:
    data = json.load(read_file)

df = json_to_dataframe_3(data)

df

,dataset,split,id,model,instruction,cot_trigger,generated_cot,correct_answer
0,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,"'name': 'command-xlarge-nightly', 'temperature...",None,None,The main purpose of farmers is to supply food.,True
1,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,"'name': 'text-davinci-003', 'temperature': 0, ...",None,None,E) supply food,True
2,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,"'name': 'flan-T5-xxl', 'temperature': 0, 'max_...",None,None,E,True
3,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,"'name': 'gpt-3.5-turbo', 'temperature': 0, 'ma...",None,None,E) supply food,True
4,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,"'name': 'text-davinci-002', 'temperature': 0, ...",None,None,\nE) supply food,True
...,...,...,...,...,...,...,...,...
11875,worldtree,test,worldtree_test_1659,"'name': 'flan-T5-xxl', 'temperature': 0, 'max_...",None,zhou-01,C is the dependent variable. The dependent var...,True
11876,worldtree,test,worldtree_test_1659,"'name': 'text-davinci-003', 'temperature': 0, ...",None,zhou-01,\n\nStep 1: The vertical axis on a graph is t...,False
11877,worldtree,test,worldtree_test_1659,"'name': 'text-davinci-002', 'temperature': 0, ...",None,zhou-01,\n\nThe first thing we need to do is identify...,True
11878,worldtree,test,worldtree_test_1659,"'name': 'command-xlarge-nightly', 'temperature...",None,zhou-01,\nThe y-axis is the vertical axis on a graph. ...,True
